In [77]:
import heapq # Colas de prioridad (heaps)
import time # Para medir el tiempo de ejecucion

class Node: 
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state # Matriz de 9x9
        self.parent = parent # El nodo padre de donde se origina el nodo actual
        self.action = action # Action tomada desde el padre para llegar al nodo
        self.path_cost = path_cost # Costo de camino desde el nodo raiz hasta el nodo actual - g(n)

    def __lt__(self, other): # Comparar dos objetos de clase node basado en el costo
        return self.path_cost < other.path_cost

In [78]:
# Funcion auxiliar para convertir la matriz a un tuple de tuples
def tuple_conv(matrix):
    return tuple(tuple(row) for row in matrix)

def a_star_search(problem, f):
    amount_explored = 0 # Contador de nodos explorados
    node = Node(state=problem.initial) # Crea el nodo raíz con el estado inicial del problema.
    frontier = [(f(node, problem), node)] # Frontera como una cola de prioridad (f(n)) con el nodo inicial.
    heapq.heapify(frontier) # Convierte la lista frontier en una cola de prioridad (heap)
    
    tuple_init = tuple_conv(problem.initial)
    reached = {tuple_init: node} # Registrar los estados alcanzados y su nodo correspondiente.

    while frontier:
        _, node = heapq.heappop(frontier) # Extrae el nodo con el valor mínimo de f de la frontera.
        amount_explored += 1

        if problem.is_goal(node.state): # Si el estado del nodo es el estado objetivo, devuelve el nodo.
            return node, amount_explored

        for child in expand(problem, node): # Expande el nodo generando sus nodos hijos.
            s = child.state
            s_tuple = tuple_conv(s)
            if s_tuple not in reached or child.path_cost < reached[s_tuple].path_cost: # Si el estado del nodo hijo no ha sido alcanzado antes o si se alcanza con un costo de camino menor, actualiza el dict y añade el nodo hijo a la frontera.
                reached[s_tuple] = child
                heapq.heappush(frontier, (f(child, problem), child)) # Añade el nodo hijo a la frontera

    return None, 0 # Se exploran todos los nodos posibles, y no se encuentra una solución

def expand(problem, node):
    state = node.state # Estado actual
    for action in problem.actions(state): # Todas las matrices posibles que se pueden obtener al llenar una celda vacia
        succesor = problem.result(state, action) # Nuevo estado despues de aplicar la acción
        cost = node.path_cost + 1 # Calcula el costo del nuevo camino usando action_cost = 1
        yield Node(state=succesor, parent=node, action=action, path_cost=cost) # Conserva el valor y pausa la ejecución de la función,
        # Cuando se vuelva a invocar se reiniciará desde la declaración del yield

### Notas frente al planteamiento del problema

- La funcion actions() no proporciona todas las posibilidades de las celdas vacias en un estado n, debido a que es muy costoso en memoria guardar todos los nodos generados en la funcion expand(). Así que, para mejorar la eficiencia, se implementó el algoritmo MRV (Minimum Remaining Values), el cual selecciona la celda vacia con menos posibilidades de movimiento; priorizando así los nodos más "importantes" en la busqueda.

- Oficialmente la funcion heuristica es h1(n) = número de celdas vacias, sin embargo, también se implementó h2(n) = sumatoria de posibilidades de las celdas vacias, la cual muestra mayor eficiencia y rapidez en los tiempos de ejecución.

In [79]:
class Problem:
    def __init__(self, initial):
        self.initial = initial # Estado inicial
    
     # Verificación de si el número es válido en la celda (reglas de Sudoku)
    def is_valid(self, state, num, i, j):
        if num in state[i]:
            return False
        
        if any(state[row][j] == num for row in range(9)):
            return False
        
        # Calcular la esquina superior izquierda de la caja 3x3
        box_row = (i // 3) * 3
        box_col = (j // 3) * 3
        
        # Verificar todas las celdas en la caja 3x3
        for row in range(box_row, box_row + 3):
            for col in range(box_col, box_col + 3):
                if state[row][col] == num:
                    return False
                
        return True
    
    # Obtener los valores posibles para una celda, cumpliendo las reglas de Sudoku
    def get_possible_values(self, state, i, j):
        possible_values = []
        for num in range(1, 10):
            if self.is_valid(state, num, i, j):
                possible_values.append(num)
        return possible_values
    
    # Encontrar la celda con el menor numero de opciones posibles
    def min_options(self, state):
        best_cell = None
        min_options = float('inf')
        best_cell_options = []
        
        for i in range(9):
            for j in range(9):
                if state[i][j] == 0:
                    options = self.get_possible_values(state, i, j)
                    if len(options) < min_options:
                        best_cell = (i, j)
                        best_cell_options = options
                        min_options = len(options)
                        
        return best_cell, best_cell_options
    
    # Todas las acciones posibles para el estado actual, implementando MRV (Minimum Remaining Values)
    def actions(self, state):
        actions = []
        best_cell, best_cell_options = self.min_options(state)
        for num in best_cell_options:
            actions.append((best_cell[0], best_cell[1], num))
        return actions
                          
    # Estado resultante de aplicar una acción
    def result(self, state, action): 
        i, j, num = action
        
        # Crear una copia profunda del estado para evitar modificar el original
        new_state = [row[:] for row in state]
        new_state[i][j] = num
        return new_state
                    
    # def action_cost(self) -> return 1        
    
    # Heurística oficial: numero de celdas vacias
    def h1(self, state): 
        count = 0
        for i in range(9):
            for j in range(9):
                if state[i][j] == 0:
                    count += 1
        return count
    
    # Heuristica alternativa: sumatoria de posibilidades de las celdas vacias
    def h2(self, state): 
        suma = 0
        for i in range(9):
            for j in range(9):
                if state[i][j] == 0:
                    suma += len(self.get_possible_values(state, i, j))
        return suma
    
    # Verificación de si el estado es el estado objetivo
    def is_goal(self, state): 
        for i in range(9):
            for j in range(9):
                if state[i][j] == 0:
                    return False
        return True

In [80]:
# f(n) = g(n) + (h1(n) | h2(n))
def f(node, problem):
    return node.path_cost + problem.h1(node.state) 

In [82]:
# Nota: Todo lo que hace h1, h2 lo hace mejor con ~0 segundos de ejecución.

# 29 numeros iniciales -> ~0 segundos con h1
example_facil = [ 
    [0, 0, 0, 0, 0, 0, 0, 0, 8],
    [6, 0, 2, 0, 8, 0, 0, 9, 4],
    [0, 0, 4, 0, 9, 0, 0, 0, 0],
    [4, 6, 7, 0, 2, 0, 1, 0, 0],
    [1, 0, 0, 5, 0, 0, 4, 0, 0],
    [0, 2, 0, 0, 1, 8, 0, 0, 7],
    [0, 0, 0, 0, 0, 0, 9, 0, 0],
    [9, 0, 0, 0, 5, 6, 0, 0, 0],
    [0, 3, 8, 7, 4, 0, 0, 0, 6]
]

# 22 numeros iniciales -> ~10 segundos con h1
example_intermedio = [ 
    [9, 0, 0, 0, 0, 3, 0, 0, 0],
    [0, 4, 0, 1, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 0, 7, 0, 0],
    [0, 5, 0, 0, 8, 0, 0, 0, 1],
    [0, 0, 0, 0, 6, 0, 2, 4, 0],
    [7, 0, 0, 0, 0, 0, 0, 0, 3],
    [0, 0, 9, 0, 4, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 8, 0],
    [8, 0, 0, 7, 0, 5, 0, 9, 0]
]

# 21 numeros iniciales -> ~1 minuto con h1
example_dificil = [ 
    [9, 0, 0, 0, 0, 3, 0, 0, 0],
    [0, 4, 0, 1, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 0, 7, 0, 0],
    [0, 5, 0, 0, 8, 0, 0, 0, 1],
    [0, 0, 0, 0, 6, 0, 2, 0, 0],
    [7, 0, 0, 0, 0, 0, 0, 0, 3],
    [0, 0, 9, 0, 4, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 8, 0],
    [8, 0, 0, 7, 0, 5, 0, 9, 0]
]

# 20 numeros iniciales -> ~5 minutos con h1
example_muy_dificil = [ 
    [9, 0, 0, 0, 0, 3, 0, 0, 0],
    [0, 4, 0, 1, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 0, 7, 0, 0],
    [0, 5, 0, 0, 8, 0, 0, 0, 1],
    [0, 0, 0, 0, 6, 0, 2, 0, 0],
    [7, 0, 0, 0, 0, 0, 0, 0, 3],
    [0, 0, 9, 0, 4, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 8, 0],
    [8, 0, 0, 7, 0, 5, 0, 9, 0]
]

# 3 numeros iniciales -> ~0 segundos con h2 (con h1 se llena la memoria)
example_casi_imposible = [ 
    [0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 7, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]
]

# Seleccionar el Sudoku a resolver
problem = Problem(example_facil)

# Busqueda A* y calcular tiempo de ejecucion
start_time = time.time()
solution, amount_explored = a_star_search(problem, f)
end_time = time.time()
execution_time = end_time - start_time

# Imprimir la solución
if solution:
    path = []
    while solution:
        path.append(solution)
        solution = solution.parent
    path.reverse()
    
    for step in range(len(path)):
        if path[step].action:
            x, y, value = path[step].action
            print(f"\nStep {step} - Placed {value} at position ({x}, {y})")
        else:
            print(f"\nPaso {step} - No action")
            
        for row in path[step].state:
            print(row)
    print("\nNúmero total de movimientos:", len(path) - 1)
    print("Número total de nodos explorados:", amount_explored)
    print("Tiempo de ejecución:", round(execution_time, 2), "segundos")
else:
    print("No solution found")


Paso 0 - No action
[0, 0, 0, 0, 0, 0, 0, 0, 8]
[6, 0, 2, 0, 8, 0, 0, 9, 4]
[0, 0, 4, 0, 9, 0, 0, 0, 0]
[4, 6, 7, 0, 2, 0, 1, 0, 0]
[1, 0, 0, 5, 0, 0, 4, 0, 0]
[0, 2, 0, 0, 1, 8, 0, 0, 7]
[0, 0, 0, 0, 0, 0, 9, 0, 0]
[9, 0, 0, 0, 5, 6, 0, 0, 0]
[0, 3, 8, 7, 4, 0, 0, 0, 6]

Step 1 - Placed 3 at position (6, 4)
[0, 0, 0, 0, 0, 0, 0, 0, 8]
[6, 0, 2, 0, 8, 0, 0, 9, 4]
[0, 0, 4, 0, 9, 0, 0, 0, 0]
[4, 6, 7, 0, 2, 0, 1, 0, 0]
[1, 0, 0, 5, 0, 0, 4, 0, 0]
[0, 2, 0, 0, 1, 8, 0, 0, 7]
[0, 0, 0, 0, 3, 0, 9, 0, 0]
[9, 0, 0, 0, 5, 6, 0, 0, 0]
[0, 3, 8, 7, 4, 0, 0, 0, 6]

Step 2 - Placed 1 at position (7, 2)
[0, 0, 0, 0, 0, 0, 0, 0, 8]
[6, 0, 2, 0, 8, 0, 0, 9, 4]
[0, 0, 4, 0, 9, 0, 0, 0, 0]
[4, 6, 7, 0, 2, 0, 1, 0, 0]
[1, 0, 0, 5, 0, 0, 4, 0, 0]
[0, 2, 0, 0, 1, 8, 0, 0, 7]
[0, 0, 0, 0, 3, 0, 9, 0, 0]
[9, 0, 1, 0, 5, 6, 0, 0, 0]
[0, 3, 8, 7, 4, 0, 0, 0, 6]

Step 3 - Placed 7 at position (0, 4)
[0, 0, 0, 0, 7, 0, 0, 0, 8]
[6, 0, 2, 0, 8, 0, 0, 9, 4]
[0, 0, 4, 0, 9, 0, 0, 0, 0]
[4, 6, 7, 0, 2, 0, 1, 0, 0